In [1]:
# import keras

from MobileNet_SSD import build_model
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
# from ssd_utils import *
import tensorflow as tf

tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [2]:
model = build_model(input_shape=(512, 512, 3), classes=21)

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 256, 256, 32) 128         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu (ReLU)               (None, 256, 256, 32) 0           conv1_bn[0][0]                   
______________________________________________________________________________________________

In [3]:
model.save('ssd_model_new.h5')

In [4]:
from tensorflow.keras.preprocessing import image
# img = Image.open('F:\\GTSDB\\00003.ppm')
# img = Image.open('F:\\GTSDB\\00000.ppm')
img = Image.open('F:\\VOC2012\\JPEGImages\\2007_005695.jpg')
img = img.resize((512, 512), Image.ANTIALIAS)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
# img = img.dot([0.299, 0.587, 0.114])


print(img.shape)


(1, 512, 512, 3)


In [5]:
pre = model.predict(img)

d:\python_3.6.7\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3508: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


In [6]:
print(pre)

[[[-1.0989228e-15 -5.6053875e-15  1.5477393e-15 ...  1.0000000e-01
    2.0000000e-01  2.0000000e-01]
  [ 2.6796222e-15  3.2524113e-15  1.6380029e-15 ...  1.0000000e-01
    2.0000000e-01  2.0000000e-01]
  [ 3.8787248e-16  4.9963424e-15 -1.7573646e-15 ...  1.0000000e-01
    2.0000000e-01  2.0000000e-01]
  ...
  [-1.2755532e-19 -1.2596224e-19  5.0645509e-20 ...  1.0000000e-01
    2.0000000e-01  2.0000000e-01]
  [ 1.9688006e-19 -3.6763713e-20  2.0287326e-19 ...  1.0000000e-01
    2.0000000e-01  2.0000000e-01]
  [ 4.8856479e-20  9.9319072e-20  1.1358349e-19 ...  1.0000000e-01
    2.0000000e-01  2.0000000e-01]]]


In [7]:
# print(pre[0, :, -8:][0])
import pickle

pickle.dump(pre[0, :, -8:], open('prior_boxes_VOC_512_mobileNet_ssd.pkl', 'wb'))

In [3]:
import pickle
prior = pickle.load(open('prior_boxes_ssd300.pkl', 'rb'))

print(prior[0:3])

[[0.         0.         0.06315789 0.06315789 0.1        0.1
  0.2        0.2       ]
 [0.         0.         0.08386857 0.04851323 0.1        0.1
  0.2        0.2       ]
 [0.         0.         0.04851323 0.08386857 0.1        0.1
  0.2        0.2       ]]


In [6]:
import pickle

gt = pickle.load(open('gt_pascal.pkl', 'rb'))
print(gt['frame02089.png'])
# print(gt['2007_000027.jpg'])

[[0.5625     0.09861111 0.790625   0.46805556 1.         0.
  0.        ]
 [0.403125   0.38055556 0.509375   0.81944444 1.         0.
  0.        ]]


In [4]:
from get_data_from_xml import XML_preprocessor

preprocessor = XML_preprocessor('F:\\VOC2012\\Annotations\\')

preprocessor_data = preprocessor.data

In [5]:
preprocessor_data['2007_000027.jpg']

array([[0.6563786 , 0.614     , 0.69958848, 0.652     , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [3]:
from ssd_utils import BBoxUtility

bbox = BBoxUtility(4, prior)

In [26]:
from utils import Generator

keys = sorted(gt.keys())
num_train = int(round(0.8 * len(keys)))
train_keys = keys[:num_train]
val_keys = keys[num_train:]
num_val = len(val_keys)

gen = Generator(gt, bbox, 16, '../../frames/',
                train_keys, val_keys,
                (300, 300), do_crop=False)

In [27]:
g = gen.generate()

In [28]:
n = g.__next__()

In [2]:
m = keras.models.load_model('../model/VOC/VOC_MobileNet_53.50.h5')

m.summary()

from keras.applications import mobilenet
mobilenet.MobileNet()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 150, 150, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 150, 150, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 150, 150, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 150, 150, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 150, 150, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 150, 150, 32)      0     

In [4]:
model.load_weights('../model/VOC/VOC_MobileNet_53.50.h5', by_name=True)

In [7]:
freeze = [
    'input_1', 'conv1', 'conv1_bn', 'conv1_relu',
    'conv_dw_1', 'conv_dw_1_bn', 'conv_dw_1_relu', 'conv_pw_1', 'conv_pw_1_bn', 'conv_pw_1_relu',
    'conv_pad_2', 'conv_dw_2', 'conv_dw_2_bn', 'conv_dw_2_relu', 'conv_pw_2', 'conv_pw_2_bn', 'conv_pw_2_relu',
    'conv_dw_3', 'conv_dw_3_bn', 'conv_dw_3_relu', 'conv_pw_3', 'conv_pw_3_bn', 'conv_pw_3_relu',
    'conv_pad_4', 'conv_dw_4', 'conv_dw_4_bn', 'conv_dw_4_relu', 'conv_pw_4', 'conv_pw_4_bn', 'conv_pw_4_relu',
    'conv_dw_5', 'conv_dw_5_bn', 'conv_dw_5_relu', 'conv_pw_5', 'conv_pw_5_bn', 'conv_pw_5_relu',
    'conv_pad_6', 'conv_dw_6', 'conv_dw_6_bn', 'conv_dw_6_relu', 'conv_pw_6', 'conv_pw_6_bn', 'conv_pw_6_relu',
    'conv_dw_7', 'conv_dw_7_bn', 'conv_dw_7_relu', 'conv_pw_7', 'conv_pw_7_bn', 'conv_pw_7_relu',
    'conv_dw_8', 'conv_dw_8_bn', 'conv_dw_8_relu', 'conv_pw_8', 'conv_pw_8_bn', 'conv_pw_8_relu',
    'conv_dw_9', 'conv_dw_9_bn', 'conv_dw_9_relu', 'conv_pw_9', 'conv_pw_9_bn', 'conv_pw_9_relu',
    'conv_dw_10', 'conv_dw_10_bn', 'conv_dw_10_relu', 'conv_pw_10', 'conv_pw_10_bn', 'conv_pw_10_relu',
    'conv_dw_11', 'conv_dw_11_bn', 'conv_dw_11_relu', 'conv_pw_11', 'conv_pw_11_bn', 'conv_pw_11_relu',
    'conv_pad_12', 'conv_dw_12', 'conv_dw_12_bn', 'conv_dw_12_relu', 'conv_pw_12', 'conv_pw_12_bn', 'conv_pw_12_relu',
    'conv_dw_13', 'conv_dw_13_bn', 'conv_dw_13_relu', 'conv_pw_13', 'conv_pw_13_bn', 'conv_pw_13_relu'
]

for L in model.layers:
    if L.name in freeze:
        L.trainable = False

input_1
conv1
conv1_bn
conv1_relu
conv_dw_1
conv_dw_1_bn
conv_dw_1_relu
conv_pw_1
conv_pw_1_bn
conv_pw_1_relu
conv_pad_2
conv_dw_2
conv_dw_2_bn
conv_dw_2_relu
conv_pw_2
conv_pw_2_bn
conv_pw_2_relu
conv_dw_3
conv_dw_3_bn
conv_dw_3_relu
conv_pw_3
conv_pw_3_bn
conv_pw_3_relu
conv_pad_4
conv_dw_4
conv_dw_4_bn
conv_dw_4_relu
conv_pw_4
conv_pw_4_bn
conv_pw_4_relu
conv_dw_5
conv_dw_5_bn
conv_dw_5_relu
conv_pw_5
conv_pw_5_bn
conv_pw_5_relu
conv_pad_6
conv_dw_6
conv_dw_6_bn
conv_dw_6_relu
conv_pw_6
conv_pw_6_bn
conv_pw_6_relu
conv_dw_7
conv_dw_7_bn
conv_dw_7_relu
conv_pw_7
conv_pw_7_bn
conv_pw_7_relu
conv_dw_8
conv_dw_8_bn
conv_dw_8_relu
conv_pw_8
conv_pw_8_bn
conv_pw_8_relu
conv_dw_9
conv_dw_9_bn
conv_dw_9_relu
conv_pw_9
conv_pw_9_bn
conv_pw_9_relu
conv_dw_10
conv_dw_10_bn
conv_dw_10_relu
conv_pw_10
conv_pw_10_bn
conv_pw_10_relu
conv_dw_11
conv_dw_11_bn
conv_dw_11_relu
conv_pw_11
conv_pw_11_bn
conv_pw_11_relu
conv_pad_12
conv_dw_12
conv_dw_12_bn
conv_dw_12_relu
conv_pw_12
conv_pw_12_bn
conv_pw

In [1]:
from VGG_SSD import SSD300

VGG_model = SSD300((300, 300, 3), 21)

In [2]:
VGG_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 300, 300, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 300, 300, 64) 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 150, 150, 64) 0           conv1_2[0][0]                    
______________________________________________________________________________________________